In [ ]:
from pathlib import Path

import numpy as np
import scipy
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

import librosa
librosa.show_versions()

### build DataFrame of all file paths

In [ ]:
# ENTER THE PATH TO YOUR GTZAN DIRECTORY HERE
gtzan_base_path = Path('path/to/gtzan/Data/genres_original')

In [ ]:
col_genre = []
col_file = []
col_path = []
for genre_path in gtzan_base_path.iterdir():
    genre = genre_path.name
    for file_path in genre_path.iterdir():
        file = file_path.name
        col_genre.append(genre)
        col_file.append(file)
        col_path.append(file_path)

files = pd.DataFrame(index=pd.Index(data=col_file, name='file'), data={'genre': col_genre, 'path': col_path})

In [ ]:
files.head(5)

## feature extraction functions

In [ ]:
def low_energy_rate(y, frame_length):
    frame_energies = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=frame_length, center=False)[0]
    mean_energy = np.mean(frame_energies)
    return sum(frame_energies < mean_energy) / len(frame_energies)

In [ ]:
def spectral_flux(y, frame_length):
    frame_stft = abs(librosa.stft(y=y, n_fft=frame_length, hop_length=frame_length, center=False))
    # normalise spectrum per frame (catch case where frame only has silence)
    frame_stft_max = np.max(frame_stft, axis=0)
    frame_stft_max[frame_stft_max == 0] = 1
    frame_stft /= frame_stft_max

    spectral_diff = frame_stft[:,:-1] - frame_stft[:,1:]
    spectral_flux = np.linalg.norm(spectral_diff, axis=0)
    return spectral_flux

In [ ]:
def rhythm_features(y, sr, hop_length):
    # calculate onset autocorrelation
    onset_envelope = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length, center=False)
    max_lag = 2 * sr // hop_length # 2sec = 30bpm, as we are only considering peaks between 40 and 200 bpm anyway
    onset_autocorr = librosa.util.normalize(librosa.autocorrelate(onset_envelope, max_size=max_lag))
    freqs = librosa.tempo_frequencies(onset_autocorr.shape[0], hop_length=hop_length, sr=sr)

    # find overall sum of tempogram between 40 and 200 bpm
    autocorr_freqs = list(filter(lambda x: 40 <= x[0] <= 200, zip(freqs, onset_autocorr)))
    autocorr_sum = sum(x[1] for x in autocorr_freqs)

    # find peaks of tempogram and corresponding frequencies
    peak_indices, props = scipy.signal.find_peaks(onset_autocorr, height=0)
    peaks = [[freqs[i], props['peak_heights'][n]] for n, i in enumerate(peak_indices)]

    # limit to range of 40-200 bpm and sort by peak height
    peaks = sorted(filter(lambda x: 40 <= x[0] <= 200, peaks), key=lambda x: x[1], reverse=True)
    freq_0, amp_0 = peaks[0]
    freq_1, amp_1 = peaks[1]

    # calculate beat salience
    # get peaks that are on (or one off) a multiple of the tempo (highest peak)
    beat_salience_peaks = []
    for i in peak_indices:
        for j in [i-1, i, i+1]:
            if freqs[j] in np.array([1/5, 1/4, 1/3, 1/2, 1, 2, 3, 4, 5]) * freq_0: # up to factor 5 (40 vs. 200 bpm)
                beat_salience_peaks.append([freqs[j], onset_autocorr[j]])
    # calculate log difference from 100 bpm and select closest peak
    beat_salience_peaks = [[abs(np.log(freq) - np.log(100)), amp] for freq, amp in beat_salience_peaks]
    beat_salience = sorted(beat_salience_peaks, key=lambda x: x[0])[0][1]

    return amp_0 / autocorr_sum, amp_1 / autocorr_sum, amp_1 / amp_0, freq_0, freq_1, autocorr_sum, beat_salience

In [ ]:
def tonal_features(y, sr, frame_length, hop_length):
    # calculate chromagram and sum over all frames to get total amplitude per pitch class
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, n_fft=frame_length, hop_length=hop_length, tuning=0, center=False, norm=None)
    chromagram_sum = np.sum(chroma)
    chroma = [(pitch_class, amplitude) for pitch_class, amplitude in enumerate(np.sum(chroma, axis=1))]

    # get pitch class and amplitude of highest peak, and pitch interval to second highest peak
    peak_0, peak_1 = sorted(chroma, key=lambda x: x[1], reverse=True)[:2]
    amp_0 = peak_0[1] / chromagram_sum
    pitch_0 = (peak_0[0] * 7) % 12
    pitch_interval = ((peak_0[0] * 7) % 12) - ((peak_1[0] * 7) % 12)
    
    return amp_0, pitch_0, pitch_interval, chromagram_sum

### playground

In [ ]:
path = files.iloc[300].path
print(path)
x, sr = librosa.load(path)
librosa.display.waveshow(x[100:200], sr=sr)
print(len(x))

In [ ]:
plt.figure(figsize=(20,5))
plt.xlim(0, 5)
librosa.display.waveshow(x, sr=sr, alpha=0.4)

In [ ]:
plt.figure(figsize=(20,5))
plt.xlim(0, 5)
S = np.abs(librosa.stft(y=x, n_fft=1024, hop_length=1024, center=False))
librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), sr=sr, y_axis='log', x_axis='time', alpha=0.5)

c = librosa.feature.spectral_centroid(y=x, sr=sr, n_fft=1024, hop_length=1024, center=False)[0]
c_minmax = (c - np.min(c))/(np.max(c) - np.min(c))

r = librosa.feature.spectral_rolloff(y=x, sr=sr, n_fft=1024, hop_length=1024, center=False)[0]
r_minmax = (r - np.min(r))/(np.max(r) - np.min(r))

t = librosa.frames_to_time(np.array(range(len(c))), sr=sr*2, hop_length=1024)
plt.plot(t, c, color='g')
plt.plot(t, r, color='r')

In [ ]:
plt.figure(figsize=(20,5))
plt.xlim(0, 5)
mfcc = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=5, n_fft=1024, hop_length=1024, center=False)
librosa.display.specshow(mfcc, x_axis='time')

print(np.mean(mfcc, axis=1))
print(np.var(mfcc, axis=1))

In [ ]:
plt.figure(figsize=(20,5))
plt.xlim(0, 5)
chroma = librosa.feature.chroma_stft(y=x, sr=sr, hop_length=1024, tuning=0, center=False, norm=None)
print(np.sum(chroma, axis=1))
# NOTE: tuning=0 must be specified, as librosa.core.estimate_tuning is apparently bugged and crashes the kernel
librosa.display.specshow(chroma, y_axis='chroma', x_axis='time')

In [ ]:
plt.figure(figsize=(20,5))
plt.xlim(0, 5)

librosa.display.waveshow(x, sr=sr, alpha=0.4)

o_env = librosa.onset.onset_strength(y=x, sr=sr, center=False)

C = np.abs(librosa.cqt(y=x,sr=sr))
o_env_cqt = librosa.onset.onset_strength(sr=sr, S=librosa.amplitude_to_db(C, ref=np.max))

t = librosa.frames_to_time(np.array(range(len(o_env))), sr=sr, hop_length=512)
plt.plot(t, o_env/np.max(o_env), label='mel')
plt.plot(t, o_env_cqt/np.max(o_env_cqt), label='cqt')
plt.legend()

In [ ]:
plt.figure(figsize=(20,5))
hop_length=128
o_env = librosa.onset.onset_strength(y=x, sr=sr, hop_length=hop_length, center=False)
tempogram = librosa.feature.tempogram(onset_envelope=o_env, sr=sr, hop_length=hop_length)
tempogram = np.mean(tempogram, axis=1)
max_lag = 2 * sr // hop_length
ac_global = librosa.autocorrelate(o_env, max_size=max_lag)
ac_global = librosa.util.normalize(ac_global)
tempo = librosa.feature.tempo(onset_envelope=o_env, sr=sr, hop_length=hop_length)[0]
print(tempo)
freqs_autocorr = librosa.tempo_frequencies(ac_global.shape[0], hop_length=hop_length, sr=sr)
freqs_tempogram = librosa.tempo_frequencies(tempogram.shape[0], hop_length=hop_length, sr=sr)
plt.xlim((40,200))
plt.xlabel('bpm')
plt.plot(freqs_autocorr[1:], ac_global[1:])
plt.plot(freqs_tempogram[1:], tempogram[1:])

In [ ]:
# figure 4.2
plt.figure(figsize=(10,3.333))
plt.xlim((40, 200))
plt.xlabel('Tempo (bpm)')
plt.ylim((0, 1))
plt.ylabel('normalisierte Autokorrelation')
plt.plot(freqs_autocorr[1:], ac_global[1:], label='Autokorrelation für disco.00000.wav')
plt.plot([114.84, 114.84], [0, 1], '--', color='black', label='Peak 1 (114.84 bpm)')
plt.plot([57.42, 57.42], [0, 1], '-.', color='black', label='Peak 2 (57.42 bpm)')
plt.legend()
plt.savefig('autocorrelation.pdf', bbox_inches='tight')

### calculate features

In [ ]:
features = [                   # TIMBRE FEATURES
            'spec_cent_mean',  # spectral centroid mean and variance
            'spec_cent_var',
            'spec_roll_mean',  # spectral rolloff mean and variance
            'spec_roll_var',
            'spec_flux_mean',  # spectral flux mean and variance
            'spec_flux_var',
            'zcr_mean',        # zero crossing rate mean and variance
            'zcr_var',
            'low_energy_rate', # low energy rate
            'mfcc_1_mean',     # mean and variance of the first five MFCCs
            'mfcc_2_mean',
            'mfcc_3_mean',
            'mfcc_4_mean',
            'mfcc_5_mean',
            'mfcc_1_var',
            'mfcc_2_var',
            'mfcc_3_var',
            'mfcc_4_var',
            'mfcc_5_var',
                               # RHYTHM FEATURES
            'rel_amp_peak_0',  # relative amplitudes of the two highest peaks between 40 and 200 bmp in tempogram
            'rel_amp_peak_1',
            'amp_ratio',       # ratio of the amplitudes of the two peaks
            'freq_peak_0',     # frequencies in bpm of the two peaks
            'freq_peak_1',
            'tempogram_sum',   # sum of the tempogram amplitudes between 40 and 200 bpm
            'beat_salience',   # beat salience
                               # TONAL FEATURES
            'pitch_amp',       # relative amplitude of the highest peak in the chromagram
            'pitch_class',     # pitch class of the highest peak
            'pitch_interval',  # interval between two highest peaks in the chromagram (distance on circle of fifths)
            'chromagram_sum',  # sum of all chromagram amplitudes
           ]

multiindex = pd.MultiIndex.from_product([files.index, ['full', 'start', 'middle', 'end']], names=['file', 'section'])
features = pd.DataFrame(index=multiindex, columns=features)

frame_length = 1024

for i, (index, row) in enumerate(files.iterrows()):
    file = index
    y, sr = librosa.load(row.path)
    section = 'full'

    # calculate section boundaries (inclusive)
    l = len(y)
    sections = [
        ('full',   0,      l - 1),
        ('start',  0,      l//3 - 1),
        ('middle', l//3,   2*l//3 - 1),
        ('end',    2*l//3, l - 1)
    ]

    for s in sections:
        section = s[0]
        y_section = y[s[1] : s[2] + 1]
        
        #### SPECTRAL CENTROID
        spec_cent = librosa.feature.spectral_centroid(y=y_section, sr=sr, n_fft=frame_length, hop_length=frame_length, center=False)[0]
        features.loc[(file, section), 'spec_cent_mean':'spec_cent_var'] = np.mean(spec_cent), np.var(spec_cent)

        #### SPECTRAL ROLLOFF
        spec_roll = librosa.feature.spectral_rolloff(y=y_section, sr=sr, n_fft=frame_length, hop_length=frame_length, center=False)[0]
        features.loc[(file, section), 'spec_roll_mean':'spec_roll_var'] = np.mean(spec_roll), np.var(spec_roll)

        #### SPECTRAL FLUX
        spec_flux = spectral_flux(y=y_section, frame_length=frame_length)
        features.loc[(file, section), 'spec_flux_mean':'spec_flux_var'] = np.mean(spec_flux), np.var(spec_flux)
    
        #### ZERO CROSSING RATE
        zcr = librosa.feature.zero_crossing_rate(y=y_section, frame_length=frame_length, hop_length=frame_length, center=False)
        features.loc[(file, section), 'zcr_mean':'zcr_var'] = np.mean(zcr), np.var(zcr)

        #### LOW ENERGY RATE
        ler = low_energy_rate(y=y_section, frame_length=frame_length)
        features.loc[(file, section), 'low_energy_rate'] = ler

        #### MFCC
        mfcc = librosa.feature.mfcc(y=y_section, sr=sr, n_mfcc=5, n_fft=frame_length, hop_length=frame_length, center=False)
        features.loc[(file, section), 'mfcc_1_mean':'mfcc_5_mean'] = np.mean(mfcc, axis=1)
        features.loc[(file, section), 'mfcc_1_var':'mfcc_5_var'] = np.var(mfcc, axis=1)    

        #### RHYTHM FEATURES
        amp_0, amp_1, amp_r, freq_0, freq_1, tempogram_sum, beat_salience = rhythm_features(y=y_section, sr=sr, hop_length=128)
        features.loc[(file, section), 'rel_amp_peak_0':'beat_salience'] = amp_0, amp_1, amp_r, freq_0, freq_1, tempogram_sum, beat_salience

        #### TONAL FEATURES
        amp_0, pitch_0, pitch_interval, chromagram_sum = tonal_features(y=y_section, sr=sr, frame_length=frame_length, hop_length=frame_length)
        features.loc[(file, section), 'pitch_amp':'chromagram_sum'] = amp_0, pitch_0, pitch_interval, chromagram_sum
    
    #### PRINT PROGRESS
    if (i+1)%5 == 0:
        print(f'Processed file {i+1} of 1000 ({(i+1)/10}%)', end='\r')

print()

In [ ]:
features.head(20)

### save DataFrame to csv

In [ ]:
full_df = features.join(files).drop(columns=['path'])
full_df.head(10)

In [ ]:
output_filename = 'gtzan_features.csv'
full_df.to_csv(output_filename)

### normalise features and generate dataset splits

In [ ]:
features_normalised = (features - features.min()) / (features.max() - features.min())
features_normalised = features_normalised.join(files).drop(columns=['path'])

In [ ]:
features_normalised['split'] = features_normalised.apply(lambda x: int(x.name[0].split('.')[1]) % 5, axis=1)
features_normalised['split'].xs('full', level='section').head(12)

In [ ]:
features_normalised

In [ ]:
Path('./training_data').mkdir(exist_ok=True)
for split in range(5):
    df_train = features_normalised[features_normalised['split'] != split].drop(columns=['split'])
    df_test  = features_normalised[features_normalised['split'] == split].drop(columns=['split'])

    df_train.to_csv(f'training_data/train_{split}.csv')
    df_test.to_csv(f'training_data/test_{split}.csv')